In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from gensim.models import Word2Vec, KeyedVectors


In [2]:
# Playlists as sentences, ids as words

df_playlists = pd.read_csv('data/playlist.tsv', sep='\t')

playlists = df_playlists.track_uris.apply(eval).tolist()

In [3]:
model = Word2Vec(sentences=playlists, vector_size=128, window=100, min_count=1, workers=8)
model.save("word2vec128.model")

In [4]:
model = Word2Vec.load("word2vec128.model")
word_vectors = model.wv
word_vectors.save("word2vec128.wordvectors")

In [5]:
wv = KeyedVectors.load("word2vec128.wordvectors", mmap='r')

In [6]:
wv['0UaMYEvWZi0ZqiDOoHU3YI']

memmap([-1.7170135e+00, -2.3962460e+00,  6.5327615e-01,  1.4792818e+00,
        -4.1627612e+00, -3.7363815e+00, -4.0420847e+00,  5.5401316e+00,
         3.6785965e+00, -8.0635004e+00, -2.8311641e+00, -3.8945107e+00,
        -3.0691066e-01,  6.4100319e-01, -8.6864406e-01, -3.9300735e+00,
         5.5363994e+00, -2.9236808e+00,  5.6453266e+00,  2.1363355e-03,
        -3.5578997e+00,  3.2450306e+00,  1.3382255e+00, -3.0441615e-01,
         2.0134881e+00, -4.1783738e+00,  7.8383976e-01, -8.2223302e-01,
         1.3025694e+00,  7.4136597e-01,  7.2722640e+00,  3.9815881e+00,
         2.9444141e+00,  5.9522386e+00,  2.3802121e+00, -2.0279431e+00,
         4.1253424e+00, -1.7454195e-01, -9.4524580e-01, -3.1904263e+00,
        -4.7110631e-03,  8.5059285e-01, -6.3117027e-01,  3.9513373e+00,
         1.6436853e+00,  3.4933012e+00,  1.4838673e+00, -2.1465943e+00,
        -1.5491265e+00, -4.3947349e+00,  7.1400744e-01, -5.3090148e+00,
         1.5631786e-01,  6.2345719e+00, -3.4194691e+00,  2.13700

Mappeamos los tracks a vectores usando W2V, siguiendo [el siguiente paper](https://arxiv.org/pdf/2304.12257). Ahora debemos hacer el entrenamiento contrastivo sobre estos vectores, tomando en cuenta su coocurrencia en playlists (como menciona el paper), pero ademas la similitud de sus features de audio.